In [1]:
!pip install kagglehub
!pip install tensorflow tensorflow-io opencv-python matplotlib scikit-learn



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/22.9 MB ? eta -:--:--
   ----------- ---------------------------- 6.3/22.9 MB 43.1 MB/s eta 0:00:01
   ----------- ---------------------------- 6.3/22.9 MB 43.1 MB/s eta 0:00:01
   ----------- ---------------------------- 6.3/22.9 MB 43.1 MB/s eta 0:00:01
   ----------- ---------------------------- 6.3/22.9 MB 43.1 MB/s eta 0:00:01
   -------------- ------------------------- 8.4/22.9 MB 8.0 MB/s eta 0:00:02
   ---------------- ----------------------- 9.4/22.9 MB 8.4 MB/s eta 0:00:02
   ---------------- ----------------------- 9.4/22.9 MB 8.4 MB/s eta 0:00:02
   ---------------- ----------------------- 9.4/22.9 MB 8.4 MB/s eta 0:00:02
   ---------------- ----------------------- 9.4/22.9 MB 8.4 MB/s eta 0:00:02
   ---------------- ----------------------- 9.4/22.9 MB 8.4 MB/s eta 0:00:02
   ---------------- ----------------------- 9.4/22.9 MB 8.4 MB/s eta 0:00:02
   ------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import kagglehub

path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")
print(path)


c:\Users\panka\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 2.70G/2.70G [03:21<00:00, 14.4MB/s] 

Extracting files...


C:\Users\panka\.cache\kagglehub\datasets\vipoooool\new-plant-diseases-dataset\versions\2


In [3]:
data_dir = f"{path}/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)"
train_dir = f"{data_dir}/train"
val_dir   = f"{data_dir}/valid"
test_dir  = f"{path}/test"

In [4]:
import tensorflow as tf

img_size = (224, 224)
batch_size = 32

AUTOTUNE = tf.data.AUTOTUNE

def configure(ds):
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

# Create initial datasets
train_ds_raw = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

val_ds_raw = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

test_ds_raw = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

# Capture class names before transformations
num_classes = len(train_ds_raw.class_names)

# Apply configure and preprocess to the datasets
train_ds = configure(train_ds_raw)
val_ds = configure(val_ds_raw)
test_ds = configure(test_ds_raw)


c:\Users\panka\AppData\Local\Programs\Python\Python310\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.
Found 33 files belonging to 1 classes.
Found 33 files belonging to 1 classes.


Let's inspect the contents of the `test_dir` to verify its structure.

In [5]:
import os
print(os.listdir(test_dir))

['test']


Let's first inspect the contents of the downloaded dataset directory to understand its structure.

In [6]:
import os

# List the contents of the downloaded dataset directory
print(os.listdir(path))

['New Plant Diseases Dataset(Augmented)', 'test']


In [7]:
import os
print(os.listdir(data_dir))

['train', 'valid']


In [8]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models

base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
base.trainable = False

model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation="softmax")
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 38)             │        48,678 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,098,249 (15.63 MB)

 Trainable params: 48,678 (190.15 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [9]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
  25/2197 ━━━━━━━━━━━━━━━━━━━━ 11:58 331ms/step - accuracy: 0.0579 - loss: 3.6092

KeyboardInterrupt: 

In [43]:
model.evaluate(val_ds)


550/550 ━━━━━━━━━━━━━━━━━━━━ 23s 41ms/step - accuracy: 0.9800 - loss: 0.0601


[0.05874188616871834, 0.9805372357368469]

Let's inspect the contents of the parent directory of `test_dir` to find the correct path.

In [38]:
import os

parent_test_dir = os.path.dirname(test_dir)
print(f"Contents of {parent_test_dir}:")
print(os.listdir(parent_test_dir))

Contents of /kaggle/input/new-plant-diseases-dataset:
['New Plant Diseases Dataset(Augmented)', 'new plant diseases dataset(augmented)', 'test']


In [41]:
from tensorflow.keras.applications.efficientnet import preprocess_input

def preprocess(image, label):
    return preprocess_input(image), label

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)
test_ds = test_ds.map(preprocess)

In [ ]:
import os

# Save human-readable class names for inference mapping
os.makedirs("models", exist_ok=True)
with open("models/classes.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(train_ds_raw.class_names))


In [ ]:
from pathlib import Path
Path("models").mkdir(exist_ok=True)
model.save("models/plant_disease_model.keras")
print("Saved to models/plant_disease_model.keras")


In [ ]:
# Verify saved model is single-input/single-output and runnable
import numpy as np
try:
    import keras
    m = keras.saving.load_model("models/plant_disease_model.keras")
except Exception:
    from tensorflow import keras
    m = keras.models.load_model("models/plant_disease_model.keras", compile=False)

print("inputs:", len(m.inputs), "outputs:", len(m.outputs))
# Dummy prediction to ensure graph is valid
x = np.zeros((1, 224, 224, 3), dtype=np.float32)
p = m.predict(x)
print("pred shape:", p.shape)
